# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: _____

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [113]:
%%capture

#Instalando o tweepy
!pip install tweepy
import pandas as pd
import re
from emoji import UNICODE_EMOJI

ModuleNotFoundError: No module named 'emoji'

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [15]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [16]:
#Produto escolhido:
produto = 'Windows'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

Capturando os dados do twitter:

In [19]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q = produto, lang=lang, tweet_mode="extended").items():
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
            break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [20]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [84]:
Doc = pd.read_excel('Windows.xlsx')
Doc.head()

,Treinamento,Relevancia
0,@evelynchartres oh that's good to know! i have...,1
1,how to resolve a usb security key is not worki...,1
2,"kumo ind. llc korea\naluminum &amp; u-pvc, win...",0
3,full time: heritage windows 2 - 1 braishfield ...,0
4,@sagaciouszu @aciouvistas @gorydetail it doesn...,1


In [54]:
print(Doc.Treinamento[4].strip('@'))

sagaciouszu @aciouvistas @gorydetail it doesn't function well with windows core stuff. like the font will get squished but in my prototypes i use it and it makes reading my games much easier


In [117]:
limp=[]
for i in Doc.Treinamento:
    a=re.sub('@','',i)
    a=re.sub(':','',a)
    a=re.sub("'",'',a)
    a=re.sub('-','',a)
    a=re.sub('#','',a)
    a=re.sub('"','',a)
    a=re.sub('!','',a)
    a=re.sub('\n',' ',a)
    a=re.sub('\t',' ',a)
    limp.append(a)
print(limp)

['evelynchartres oh thats good to know i have windows 6 haha i really need a new computer', 'how to resolve a usb security key is not working on windows 10?  https//t.co/haugaqmie0 https//t.co/gzja1nk7ug', 'kumo ind. llc korea aluminum &amp; upvc, windows, doors, modern kitchen trading  showroom sheikh mohammed bin salem road, ras al khaimah, uae from saturday  thursday 0900am100pm then 400pm1000pm  factory umm al quwain, uae from saturday  thursday 700am  700pm https//t.co/kplxyscls4', 'full time heritage windows 2  1 braishfield football club herbra pitchero https//t.co/zl7ulrkfe8 https//t.co/ffqw0sgspv', 'sagaciouszu aciouvistas gorydetail it doesnt function well with windows core stuff. like the font will get squished but in my prototypes i use it and it makes reading my games much easier', 'blakegc windows xp', 'the f7 has been cleared and is open for traffic in both directions. motorists are still advised to keep windows rolled up while driving the pass, however.', 'maggieelmasry

In [107]:
Doc_limpo = pd.DataFrame()
Doc_limpo['Treinamento']=limp
Doc_limpo['Relevancia']=Doc.Relevancia
Doc_limpo.head()

,Treinamento,Relevancia
0,evelynchartres oh thats good to know i have wi...,1
1,how to resolve a usb security key is not worki...,1
2,"kumo ind. llc korea aluminum &amp; upvc, windo...",0
3,full time heritage windows 2 1 braishfield fo...,0
4,sagaciouszu aciouvistas gorydetail it doesnt f...,1


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**